# 🏋️ Fitness-AQA Vision Pipeline (Google Colab - Python 3.12 Fix)

This version contains a specific patch for the **Python 3.12 `ImpImporter` error** and **Numpy 2.0 incompatibility** currently affecting Colab.

---

## ⚙️ INSTRUCTIONS:
1. **Runtime → Change runtime type → GPU (T4)**
2. Run **Step 1** (Installation). It will **automatically restart** the runtime.
3. After the restart, start from **Step 2**.

---

## 📦 Step 1: Install Dependencies (Python 3.12 Patch)

**This cell will fix the `ImpImporter` error and restart the runtime.**

In [ ]:
import os
import sys

print("🚀 Starting Python 3.12 Compatibility Patch...")

# 1. Force update setuptools to fix 'ImpImporter' error
!pip install --upgrade setuptools==69.5.1 pip wheel -q

# 2. Install numpy 1.x (Colab's default 2.x breaks many CV libraries)
!pip install "numpy<2.0.0" --force-reinstall -q

# 3. Install OpenMIM and MMPose stack
!pip install -U openmim -q
!mim install mmengine -q
!mim install "mmcv>=2.1.0,<2.2.0" -q
!mim install "mmdet>=3.0.0" -q
!mim install "mmpose>=1.0.0" -q

# 4. Install signal processing dependencies
!pip install scipy opencv-python matplotlib -q

print("\n✅ Patch Applied! Restarting runtime to apply changes...")

# Force restart
os.kill(os.getpid(), 9)

## ✅ Step 2: Verify Fix

After the restart above, run this cell. If it prints "SUCCESS", the `ImpImporter` error is gone.

In [ ]:
try:
    import pkg_resources
    import numpy as np
    import scipy
    from mmpose.apis import MMPoseInferencer
    print(f"✅ SUCCESS! Python {np.sys.version.split()[0]} is now compatible.")
    print(f"✅ NumPy Version: {np.__version__} (Should be 1.2x.x)")
    print(f"✅ MMPose Loaded Successfully")
except Exception as e:
    print(f"❌ ERROR: {e}")
    print("\nTry running Step 1 again, or check Runtime -> Change runtime type is set to GPU.")

## 📤 Step 3: Upload Your Video

In [ ]:
from google.colab import files
import os

uploaded = files.upload()
if uploaded:
    video_path = list(uploaded.keys())[0]
    print(f"✅ Ready to process: {video_path}")
else:
    print("❌ No file uploaded.")

## 🔧 Step 4: Define Processor Logic

In [ ]:
import json
import logging
from scipy.signal import savgol_filter

class PoseExtractor:
    def __init__(self, mode='human', device='cuda'):
        self.inferencer = MMPoseInferencer(mode, device=device)

    def smooth_signal(self, keypoints, window_length=5, polyorder=2):
        if len(keypoints) < window_length: return keypoints
        smoothed = np.zeros_like(keypoints)
        for i in range(keypoints.shape[1]):
            smoothed[:, i, 0] = savgol_filter(keypoints[:, i, 0], window_length, polyorder)
            smoothed[:, i, 1] = savgol_filter(keypoints[:, i, 1], window_length, polyorder)
        return smoothed

    def process_video(self, video_path):
        print(f"🔍 Analyzing {video_path}...")
        result_generator = self.inferencer(video_path, return_vis=False)
        
        raw_keypoints, scores = [], []
        for result in result_generator:
            preds = result['predictions']
            if preds:
                raw_keypoints.append(preds[0]['keypoints'])
                scores.append(preds[0]['keypoint_scores'])
            else:
                raw_keypoints.append(np.zeros((17, 2)))
                scores.append(np.zeros(17))

        raw_keypoints = np.array(raw_keypoints)
        smoothed = self.smooth_signal(raw_keypoints)
        
        return {
            "video_id": os.path.basename(video_path),
            "frame_count": len(raw_keypoints),
            "raw_keypoints": raw_keypoints.tolist(),
            "smoothed_keypoints": smoothed.tolist(),
            "scores": np.array(scores).tolist()
        }

print("✅ Processor logic loaded!")

## 🚀 Step 5: Run Extraction

In [ ]:
extractor = PoseExtractor()
result = extractor.process_video(video_path)
with open('pullup_analysis.json', 'w') as f:
    json.dump(result, f)
print(f"\n✅ Done! Processed {result['frame_count']} frames.")

## 💾 Step 6: Download JSON for Vishal

In [ ]:
from google.colab import files
files.download('pullup_analysis.json')
print("✅ Downloaded! Send this file to Vishal.")